In [1]:
from evaluations import *
acc =1e-2
boundary = 'robin'
coarse_grid_resolution = 10
p = helmholtz(boundary = boundary)
k= 6.
resolution  = int(np.ceil(float(k*1.5+50)/coarse_grid_resolution)*coarse_grid_resolution)
cglob = -1j*k
cloc0 = 0
cloc1 = 0.02*(5-1j)
cloc2 = 0.0016*(8-1j)
cloc = cloc0+ cloc1*k+cloc2*k**2
mus = {'k': k, 'c_glob': cglob, 'c_loc': cloc}
gq, lq = localize_problem(p, coarse_grid_resolution, resolution, mus = mus, calT = True, calQ = True)

Loading pyMOR version 0.4.2-116-gadb475af
localizing problem


In [2]:
calculate_continuity_constant(gq, lq)
calculate_inf_sup_constant2(gq, lq)
calculate_lambda_min(gq, lq)
calculate_csis(gq,lq)
calculate_Psi_norm(gq,lq)

calculated_continuity_constant:  1.79186818672
calculated_inf_sup_constant:  0.227325649277
calculated all lambdas


/home/root_dennis/pymor_MA/src/pymor/operators/constructions.py:1002: RuntimeWarning: invalid value encountered in double_scalars
  if (np.linalg.norm(norm_squared.imag)/np.linalg.norm(norm_squared) > 1e-11):
/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:737: ComplexWarning: Casting complex values to real discards the imaginary part
  self.workd[yslice] = self.OP(self.workd[xslice])
/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:746: ComplexWarning: Casting complex values to real discards the imaginary part
  self.workd[yslice] = self.B(self.workd[xslice])
/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:741: ComplexWarning: Casting complex values to real discards the imaginary part
  self.workd[yslice] = self.OP(self.workd[xslice])


calculated csis
calculated all Psi_norms


In [3]:
d = gq["d"]
u = d.solve(mus)
bases = create_bases(gq, lq, num_testvecs=20, transfer = 'robin', target_accuracy = acc, calC = False)
ru = reconstruct_solution(gq,lq,bases)

k_prod = gq["k_product"]
norm = induced_norm(k_prod)
ls = norm(u-ru)
print ls

mybmat.py:20: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(blocks) != 2:


[  1.54371976e-08]


In [4]:
localizer = gq["localizer"]
rs1 = 0
for space in gq["spaces"]:
    ldict = lq[space]
    norm = induced_norm(ldict["range_product"])
    rs1 += norm(localizer.localize_vector_array(u, ldict["range_space"])-localizer.localize_vector_array(ru, ldict["range_space"]))
print rs1
print rs1/ls

[  2.58048514e-07]
[ 16.71602069]


In [7]:
localizer = gq["localizer"]
rs2 = 0
for space in gq["spaces"]:
    ldict = lq[space]
    norm = induced_norm(ldict["range_product"])
    lop = ldict["local_op_matrix"]
    prod = ldict["range_product"]._matrix
    Yinv = sp.factorized(prod.astype(complex))
    def mv(v):
        return lop.H.dot(Yinv(lop.dot(v)))
    M1 = LinearOperator(lop.shape, matvec = mv)

    #t = time.time()
    eigvals = sp.eigs(M1, M=prod, which = 'SM', tol = 1e-1, k=100)[0]
    #print time.time()-t
    eigvals = np.sqrt(np.abs(eigvals))
    eigvals.sort()
    print eigvals[0]
    rs2 += norm(localizer.localize_vector_array(u, ldict["range_space"])-localizer.localize_vector_array(ru, ldict["range_space"]))
print rs2
print rs2/rs1

ValueError: b is of incompatible size

In [47]:
sum = NumpyVectorArray(u.data*0)
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = pou[range_space](localizer.localize_vector_array(u, range_space))
	u_s = ldict["local_solution_robin"]
	u_dif = u_loc-u_s
	term = u_dif.data.T - B.dot(B.conj().T).dot(M_sparse.dot(u_dif.data.T))
	u_i = localizer.globalize_vector_array(NumpyVectorArray(term.T), range_space)
	sum += u_i
rs1 = gq["continuity_constant"]/gq["inf_sup_constant"]* norm(sum)
print rs1
print rs1/ls

[  1.95044259e-07]
[ 11.80324406]


In [48]:
sum = 0
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = pou[range_space](localizer.localize_vector_array(u, range_space))
	u_s = ldict["local_solution_robin"]
	u_dif = u_loc-u_s
	term = u_dif.data.T - B.dot(B.conj().T).dot(M_sparse.dot(u_dif.data.T))
	u_i = NumpyVectorArray(term.T)
	local_norm = induced_norm(ldict["range_product"])
	sum += local_norm(u_i)
rs2 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum
print rs2
print rs2/rs1

[  1.45907722e-06]
[ 7.48074937]


In [49]:
sum = 0
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	omega_star_space = ldict["omega_star_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = localizer.localize_vector_array(u, omega_star_space)
	u_s = ldict["local_sol2"]
	u_dif = u_loc-u_s
	u_i = u_dif
	T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
	maxval = operator_svd2(T1, S, M_sparse)[0][0]
	local_norm = induced_norm(ldict["omega_star_product"])
	sum += maxval*ldict["Psi_norm"]*local_norm(u_i)
rs3 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum
print rs3
print rs3/rs2

[  5.74957038e-06]
[ 3.94055249]


In [50]:
sum = 0
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	omega_star_space = ldict["omega_star_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = localizer.localize_vector_array(u, omega_star_space)
	T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
	maxval = operator_svd2(T1, S, M_sparse)[0][0]
	local_norm = induced_norm(ldict["omega_star_product"])
	sum += maxval*ldict["Psi_norm"]*local_norm(u_loc)*ldict["csi"]
rs4 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum
print rs4
print rs4/rs3

[  7.09762014e-06]
[ 1.23446096]


In [51]:
sum1 = 0
sum2 = 0
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	omega_star_space = ldict["omega_star_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = localizer.localize_vector_array(u, omega_star_space)
	T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
	maxval = operator_svd2(T1, S, M_sparse)[0][0]
	local_norm = induced_norm(ldict["omega_star_product"])
	sum1 += maxval**2*ldict["Psi_norm"]**2*ldict["csi"]**2
	sum2 += local_norm(u_loc)**2
rs5 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum1) * np.sqrt(sum2)
print rs5
print rs5/rs4

[  8.18406712e-06]
[ 1.15307201]


In [52]:
sum = 0
for space in gq["spaces"]:
	ldict = lq[space]
	basis = bases[space]
	M = ldict["range_product"]._matrix
	S = ldict["source_product"]._matrix
	M_sparse = scipy.sparse.csr_matrix(M)
	T = ldict["transfer_matrix_robin"]
	B = basis._array.T
	range_space = ldict["range_space"]
	omega_star_space = ldict["omega_star_space"]
	localizer = gq["localizer"]
	pou = gq["pou"]
	u_loc = localizer.localize_vector_array(u, omega_star_space)
	T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
	maxval = operator_svd2(T1, S, M_sparse)[0][0]
	sum += maxval**2*ldict["Psi_norm"]**2*ldict["csi"]**2
rs6 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum) *4* norm(u)
print rs6
print rs6/rs5

[  9.85440302e-06]
[ 1.20409607]


In [53]:
print np.log10(rs6/ls)

[ 2.77549853]


In [54]:
print np.log10(acc/ls)

[ 5.78186821]


In [55]:
print np.log10(acc/rs6)

[ 3.00636968]


In [64]:
print ls/norm(u)

[  6.40402246e-07]


In [63]:
print acc/(rs6/norm(u))

[ 27.76342662]


In [58]:
acc=1e-2
boundary = 'robin'
k=6.
cloc0 = 0
cloc1 = 0.02*(5-1j)
cloc2 = 0.0016*(8-1j)
p = helmholtz(boundary = boundary)
cglob = -1j*k
mus = {'k': k, 'c_glob': cglob, 'c_loc': cloc}
resolution  = int(np.ceil(float(k*1.5+50)/coarse_grid_resolution)*coarse_grid_resolution)
gq, lq = localize_problem(p, coarse_grid_resolution, resolution, mus = mus, calT = True, calQ = True)
calculate_continuity_constant(gq, lq)
calculate_inf_sup_constant2(gq, lq)	
calculate_lambda_min(gq, lq)
calculate_csis(gq,lq)	
calculate_Psi_norm(gq,lq)
d = gq["d"]
u = d.solve(mus)
bases = create_bases(gq, lq, num_testvecs=20, transfer = 'robin', target_accuracy = acc, calC = False)
rssum2 = 0
for space in gq["spaces"]:
    ldict = lq[space]
    basis = bases[space]
    M = ldict["range_product"]._matrix
    S = ldict["source_product"]._matrix
    M_sparse = scipy.sparse.csr_matrix(M)
    T = ldict["transfer_matrix_robin"]
    B = basis._array.T
    T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
    maxval = operator_svd2(T1, S, M_sparse)[0][0]
    rssum2 += maxval**2*ldict["csi"]**2*ldict["Psi_norm"]**2
ru = reconstruct_solution(gq,lq,bases)
print gq["full_norm"](u-ru)[0]/gq["full_norm"](u)[0]
print (gq["continuity_constant"]/gq["inf_sup_constant"])*4*np.sqrt(rssum2)

localizing problem
calculated_continuity_constant:  1.79186818672
calculated_inf_sup_constant:  0.227325649277
calculated all lambdas
calculated csis
calculated all Psi_norms
6.31383629446e-07
0.000345952747938
